### Instructions for activated the Jupyter kernel for the `cmip6hack-multigen` conda environment

In a Jupyterlab terminal, navigate to the `/cmip6hack-multigen/` folder and run the command:
```bash
source spinup_env.sh
```
which will create the `cmip6hack-multigen` conda environment and install it as a python kernel for jupyter.

Then, switch the kernel (drop-down menu in the top right hand corner) to cmip6hack-multigen and restart the notebook.

### Pre-processing climate model output in GCS

This notebook uses [`intake-esm`](https://intake-esm.readthedocs.io/en/latest/) to ingest and organize climate model output from various model generations and resave their time-mean fields locally.

In [1]:
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
import xskillscore as xs
import xesmf as xe
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
import intake
# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util
import preprocess as pp
import qc

/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [2]:
varnames = ['tas', 'pr', 'psl']
timeslice = slice('1981', '2010')

coarsen_size = 2

In [3]:
col_dict = pp.get_ipcc_collection()

In [4]:
mip_ids = pp.all_mip_ids

In [12]:
ens_dict = pp.load_ensembles(col_dict, varnames, timeslice=timeslice)
ens_dict

Loaded:  far tas
Loaded:  far pr
Loaded:  far psl
Loaded:  sar tas
Loaded:  sar pr
Loaded:  sar psl
Loaded:  tar tas
Skip TAR.MPIfM.MPIfM.historical.Amon.gn before datetime conflict.
Loaded:  tar pr
Skip TAR.MPIfM.MPIfM.historical.Amon.gn before datetime conflict.
Loaded:  tar psl
Skip TAR.MPIfM.MPIfM.historical.Amon.gn before datetime conflict.
Loaded:  cmip3 tas
Skip CMIP3.CSIRO-QCCCE.csiro_mk3_5.historical.Amon.gn before datetime conflict.
Loaded:  cmip3 pr
Skip CMIP3.CSIRO-QCCCE.csiro_mk3_5.historical.Amon.gn before datetime conflict.
Loaded:  cmip3 psl
Loaded:  cmip5 tas
Loaded:  cmip5 pr
Skip CMIP5.ICHEC.EC-EARTH.historical.Amon.gn before datetime conflict.
Loaded:  cmip5 psl
Loaded:  cmip6 tas
Loaded:  cmip6 pr
Loaded:  cmip6 psl



/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 34
  **blockwise_kwargs,


{'far': <xarray.Dataset>
 Dimensions:   (ensemble: 3, lat: 90, lon: 180, time: 3)
 Coordinates:
   * ensemble  (ensemble) object 'GFDL-GFDL' 'GISS-GISS' 'UKTR-UKTR'
   * lat       (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
   * lon       (lon) float64 1.0 3.0 5.0 7.0 9.0 ... 353.0 355.0 357.0 359.0
   * time      (time) datetime64[ns] 1985-01-01 1995-01-01 2005-01-01
 Data variables:
     tas       (ensemble, time, lat, lon) float64 dask.array<chunksize=(1, 3, 90, 180), meta=np.ndarray>
     pr        (ensemble, time, lat, lon) float64 dask.array<chunksize=(1, 3, 90, 180), meta=np.ndarray>,
 'sar': <xarray.Dataset>
 Dimensions:   (ensemble: 8, lat: 90, lon: 180, time: 360)
 Coordinates:
   * ensemble  (ensemble) object 'CSIRO-CSIRO-01' ... 'GFDL-GFDL-01'
   * lat       (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
   * lon       (lon) float64 1.0 3.0 5.0 7.0 9.0 ... 353.0 355.0 357.0 359.0
   * time      (time) datetime64[ns] 1981-01-01 1981-02-01 ...

#### Extracting time-mean

In [13]:
ens_dict = util.dict_func(ens_dict, xr.Dataset.mean, on_self=True, dim =['time'], keep_attrs=True)

In [14]:
ens_dict = util.dict_func(ens_dict, xr.Dataset.compute, on_self=True)

/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


#### Adding ensemble-mean to the ensemble

In [16]:
ens_dict = util.add_ens_mean(ens_dict)

In [17]:
ens_dict

{'far': <xarray.Dataset>
 Dimensions:   (ensemble: 4, lat: 90, lon: 180)
 Coordinates:
   * lat       (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
   * lon       (lon) float64 1.0 3.0 5.0 7.0 9.0 ... 353.0 355.0 357.0 359.0
   * ensemble  (ensemble) object 'ens-mean' 'GFDL-GFDL' 'GISS-GISS' 'UKTR-UKTR'
 Data variables:
     tas       (ensemble, lat, lon) float64 233.0 233.0 232.9 ... 252.7 252.7
     pr        (ensemble, lat, lon) float64 7.9e-06 7.882e-06 ... 7.471e-06
 Attributes:
     name:     far,
 'sar': <xarray.Dataset>
 Dimensions:   (ensemble: 9, lat: 90, lon: 180)
 Coordinates:
   * lat       (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
   * lon       (lon) float64 1.0 3.0 5.0 7.0 9.0 ... 353.0 355.0 357.0 359.0
   * ensemble  (ensemble) object 'ens-mean' 'CSIRO-CSIRO-01' ... 'GFDL-GFDL-01'
 Data variables:
     tas       (ensemble, lat, lon) float64 233.9 233.8 233.7 ... 257.1 257.2
     pr        (ensemble, lat, lon) float64 8.34e-06 8.312e